In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Niranjan kumar\\mldemo\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Niranjan kumar\\mldemo'

In [14]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    data_path:Path
    train_data_path: Path
    test_data_path: Path
    model_name: Path
    target_column : dict
    

In [15]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories



In [20]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_model_training_config(self) -> ModelTrainingConfig:
        config = self.config.model_training
        params = self.params.Logistic_Regression
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_training_config = ModelTrainingConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            train_data_path=config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            target_column = schema,    
    
            
            
        
        )

        return model_training_config

In [21]:
import pandas as pd
import os
from mlProject import logger
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

import joblib


In [22]:
class ModelTraining:
    def __init__(self, config = ModelTrainingConfig):
        self.config = config
        
    def train_model(self):
        data = pd.read_csv(self.config.data_path)
        train_data = pd.read_csv(self.config.train_data_path, usecols=lambda column: column != 'id')
        test_data = pd.read_csv(self.config.test_data_path, usecols=lambda column: column != 'id')
    
        
        target_column_schema =list(self.config.target_column.keys())[0]
        label_encoder = LabelEncoder()
        data[target_column_schema] = label_encoder.fit_transform(data[target_column_schema])
        train_data[target_column_schema] = label_encoder.transform(train_data[target_column_schema])
        test_data[target_column_schema] = label_encoder.transform(test_data[target_column_schema])

        train_x = train_data.drop(columns=[target_column_schema], axis = 1)
        test_x = test_data.drop(columns=[target_column_schema], axis = 1)
        train_y = train_data[target_column_schema]
        test_y = test_data[target_column_schema]
        
        lr = LogisticRegression(max_iter= 1000)
        lr.fit(train_x, train_y)
        
        joblib.dump(lr,os.path.join(self.config.root_dir, self.config.model_name))
        

        
        
        
        

In [23]:
try:
    config = ConfigurationManager()
    model_training_config = config.get_model_training_config()
    model_training = ModelTraining(config= model_training_config)
    model_training.train_model()
    
except Exception as e:
    raise e

[2024-03-11 01:26:18,778: INFO: common: yaml file: C:\Users\Niranjan kumar\mldemo\config\config.yaml loaded successfully]
[2024-03-11 01:26:18,781: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-11 01:26:18,789: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-11 01:26:18,792: INFO: common: created directory at: artifacts]
[2024-03-11 01:26:18,794: INFO: common: created directory at: artifacts/model_trainer]


c:\Users\Niranjan kumar\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
